In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../openTCGA/")

import networkx as nx
import numpy as np
import pandas as pd
import scipy.sparse as sps
import pickle
import matplotlib.pyplot as plt

from openomics import MultiOmics, Protein, MessengerRNA, MicroRNA, LncRNA
from openomics.database import *
from openomics.genomics import *

from moge.visualization.data_viz import matrix_heatmap, plot_coo_matrix
from moge.visualization.embedding_viz import visualize_embedding, plot_bokeh_graph
from moge.network.semantic_similarity import *

In [2]:
gtex = pd.read_table("/home/jonny/Bioinformatics_ExternalData/ProteinAtlas/rna_tissue_gtex.tsv")
gtex.rename(columns={"Gene":"protein_id", "Gene name": "protein_name"}, inplace=True)
gtex_nx = gtex.pivot_table(values="NX", index=["protein_name"], columns="Tissue")
gtex_nx

Tissue,adipose tissue,adrenal gland,amygdala,basal ganglia,breast,cerebellum,cerebral cortex,"cervix, uterine",colon,endometrium,...,skeletal muscle,skin,small intestine,spinal cord,spleen,stomach,testis,thyroid gland,urinary bladder,vagina
protein_name,,,,,,,,,,,,,,,,,,,,,
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0
A1CF,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,2.4,0.1,...,0.1,0.1,4.4,0.1,0.1,0.2,0.1,0.1,0.1,0.1
A2M,49.8,17.7,9.7,12.2,39.6,3.3,6.7,30.5,22.6,41.1,...,12.6,10.2,23.1,16.1,25.3,25.8,5.1,29.5,63.7,23.6
A2ML1,0.0,0.0,0.1,0.2,0.0,0.0,0.0,18.1,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,28.7
A3GALT2,0.4,0.1,0.1,0.2,0.4,1.3,0.1,0.4,0.2,0.5,...,0.2,0.0,0.1,0.2,0.8,0.3,0.0,0.4,0.2,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,13.4,9.7,5.8,6.5,12.6,9.9,5.4,11.8,9.2,14.4,...,28.9,18.0,9.7,7.4,10.4,9.8,9.0,11.2,14.5,11.2
ZYG11A,0.1,0.1,0.1,0.2,0.8,0.7,0.1,0.6,0.1,0.1,...,0.1,0.4,0.2,0.1,0.3,0.1,6.7,4.2,0.5,1.1
ZYG11B,7.8,9.1,10.8,11.1,6.5,21.7,16.1,7.7,10.4,9.9,...,49.8,6.1,5.8,8.4,4.7,6.3,6.1,8.3,9.2,7.0


In [3]:
cohort_name = "GTEx"
gtex_data = MultiOmics(cohort_name)
gtex_data.add_omic(Protein(cohort_name, file_path=gtex_nx,
                             gene_index_by="protein_name", transposed=True,
                             columns=None, genes_col_name=None))

# luad_data.build_samples()

Protein (34, 18815) , indexed by: protein_name


In [4]:
string = STRING()
biogrid = BioGRID(edge_attr=['Score', 'Throughput', 'Experimental System', 'Experimental System Type'])
disgenet = DisGeNet(path="https://www.disgenet.org/static/disgenet_ap1/files/downloads/", curated=True)
go = GeneOntology()

Fetching file from: https://stringdb-static.org/download/protein.links.v11.0/ 9606.protein.links.v11.0.txt.gz
Fetching file from: https://stringdb-static.org/download/protein.info.v11.0/ 9606.protein.info.v11.0.txt.gz
Fetching file from: https://stringdb-static.org/download/protein.sequences.v11.0/ 9606.protein.sequences.v11.0.fa.gz
STRING: ['protein1', 'protein2', 'combined_score']
Name: STRING
Type: Graph
Number of nodes: 19344
Number of edges: 5877115
Average degree: 607.6422
Fetching file from: https://downloads.thebiogrid.org/Download/BioGRID/Latest-Release/ BIOGRID-ALL-LATEST.tab2.zip


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DtypeWarning:

Columns (9,10,19,20) have mixed types.Specify dtype option on import or set low_memory=False.



BioGRID: ['#BioGRID Interaction ID', 'Entrez Gene Interactor A', 'Entrez Gene Interactor B', 'BioGRID ID Interactor A', 'BioGRID ID Interactor B', 'Systematic Name Interactor A', 'Systematic Name Interactor B', 'Official Symbol Interactor A', 'Official Symbol Interactor B', 'Synonyms Interactor A', 'Synonyms Interactor B', 'Experimental System', 'Experimental System Type', 'Author', 'Pubmed ID', 'Organism Interactor A', 'Organism Interactor B', 'Throughput', 'Score', 'Modification', 'Phenotypes', 'Qualifications', 'Tags', 'Source Database']
Name: BioGRID
Type: Graph
Number of nodes: 22161
Number of edges: 376634
Average degree:  33.9907
Fetching file from: https://www.disgenet.org/static/disgenet_ap1/files/downloads/ curated_gene_disease_associations.tsv.gz
Fetching file from: https://www.disgenet.org/static/disgenet_ap1/files/downloads/ all_gene_disease_associations.tsv.gz
DisGeNet: ['index', 'gene_name', 'disease_associations', 'score']
Fetching file from: http://purl.obolibrary.org/

In [5]:
# Protein
gtex_data.Protein.annotate_genomics(string, index="protein_name", 
                                    columns=['protein_size', 'protein_id', 'annotation'])
gtex_data.Protein.annotate_sequences(string, index="protein_name")
gtex_data.Protein.annotations["gene_name"] = gtex_data.Protein.annotations.index.get_level_values("protein_name")
gtex_data.Protein.annotate_genomics(database=go, index="gene_name", columns=['go_id'])
gtex_data.Protein.annotation_expressions = gtex_data.Protein.expressions.T

Seq protein_name collisions: 10


In [6]:
filted_go_annotations = go.filter_terms(gtex_data.Protein.annotations["go_id"].str.split("|"))
filted_go_annotations

protein_name
A1BG       [GO:0002576, GO:0003674, GO:0005576, GO:000557...
A1CF       [GO:0003727, GO:0005515, GO:0005515, GO:000551...
A2M        [GO:0001869, GO:0002020, GO:0002020, GO:000257...
A2ML1      [GO:0004867, GO:0005615, GO:0010951, GO:001095...
A3GALT2    [GO:0005975, GO:0006688, GO:0006688, GO:001602...
                                 ...                        
ZXDC       [GO:0003676, GO:0003700, GO:0005515, GO:000551...
ZYG11A                                          [GO:0031462]
ZYG11B     [GO:0006515, GO:0031462, GO:0032436, GO:003146...
ZYX        [GO:0001725, GO:0001725, GO:0003723, GO:000551...
ZZEF1                   [GO:0005509, GO:0008270, GO:0008270]
Name: go_id, Length: 18815, dtype: object

In [8]:
gtex_data.Protein.annotations["go_id"] = go.add_predecessor_terms(filted_go_annotations, return_str=True)
gtex_data.Protein.annotations

,gene_name,protein_size,protein_id,annotation,Transcript sequence,go_id
protein_name,,,,,,
A1BG,A1BG,495,9606.ENSP00000263100,Alpha-1B-glycoprotein; Immunoglobulin like dom...,MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVT...,GO:0002576|GO:0003674|GO:0005576|GO:0005576|GO...
A1CF,A1CF,602,9606.ENSP00000378868,APOBEC1 complementation factor; Essential comp...,MEAVCLGTCPEPEASMSTAIPGLKKGNNALQSIILQTLLEKENGQR...,GO:0003727|GO:0005515|GO:0005515|GO:0005515|GO...
A2M,A2M,1474,9606.ENSP00000323929,Alpha-2-macroglobulin; Is able to inhibit all ...,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,GO:0001869|GO:0002020|GO:0002020|GO:0002576|GO...
A2ML1,A2ML1,1454,9606.ENSP00000299698,Alpha-2-macroglobulin-like protein 1; Is able ...,MWAQLLLGMLALSPAIAEELPNYLVTLPARLNFPSVQKVCLDLSPG...,GO:0004867|GO:0005615|GO:0010951|GO:0010951|GO...
A3GALT2,A3GALT2,340,9606.ENSP00000475261,"Alpha-1,3-galactosyltransferase 2; Synthesizes...",MALKEGLRAWKRIFWRQILLTLGLLGLFLYGLPKFRHLEALIPMGV...,GO:0005975|GO:0006688|GO:0006688|GO:0016021|GO...
...,...,...,...,...,...,...
ZXDC,ZXDC,858,9606.ENSP00000374359,Zinc finger protein ZXDC; Cooperates with CIIT...,MDLPALLPAPTARGGQHGGGPGPLRRAPAPLGASPARRRLLLVRGP...,GO:0003676|GO:0003700|GO:0005515|GO:0005515|GO...
ZYG11A,ZYG11A,759,9606.ENSP00000360583,Protein zyg-11 homolog A; Probably acts as tar...,MVHFLHPGHTPRNIVPPDAQKDALGCCVVQEEASPYTLVNICLNVL...,GO:0031462
ZYG11B,ZYG11B,744,9606.ENSP00000294353,Protein zyg-11 homolog B; Probably acts as tar...,MPEDQAGAAMEEASPYSLLDICLNFLTTHLEKFCSARQDGTLCLQE...,GO:0006515|GO:0031462|GO:0032436|GO:0031462|GO...


In [23]:
import pickle
with open('moge/data/ppi_gtex_data.pickle', 'wb') as file:
#     gtex_data = pickle.load(file)
    pickle.dump(gtex_data, file)

In [15]:
string.network.number_of_nodes()

19344

# Build network

In [16]:
from moge.network.heterogeneous import HeterogeneousNetwork
network = HeterogeneousNetwork(multiomics=gtex_data, modalities=["Protein"])

Protein  nodes: 18815
Total nodes: 18815
Annotation columns: ['gene_name', 'protein_size', 'protein_id', 'annotation', 'Transcript sequence', 'go_id', 'omic']
INFO: Label gene_name is transformed by MultiLabelBinarizer
INFO: Label protein_size is transformed by MultiLabelBinarizer
INFO: Label protein_id is transformed by MultiLabelBinarizer
INFO: Label annotation is split by delim '\||;' transformed by MultiLabelBinarizer
INFO: Label go_id is split by delim '\||;' transformed by MultiLabelBinarizer
INFO: Label omic is transformed by MultiLabelBinarizer


In [17]:
network.add_edges(biogrid.get_interactions(network.node_list, data=True, inclusive=True), 
                  directed=False, 
                  modalities=["Protein", "Protein"], database=biogrid.name())

333187 edges added.


In [12]:
network.add_edges(string.get_interactions(network.node_list, data=True, inclusive=True), 
                  directed=False, 
                  modalities=["Protein", "Protein"], database=string.name())

5324839 edges added.


In [18]:
with open('moge/data/biogrid_gtex_network.pickle', 'wb') as file:
#     network = pickle.load(file)
    pickle.dump(network, file)

# Build Multiplex network

In [19]:
from moge.network import MultiplexAttributedNetwork
from networkx import Graph, DiGraph

multi_network = MultiplexAttributedNetwork(multiomics=gtex_data,
                                     modalities=["Protein"], 
                                     layers={
                                         (Protein.name(), Protein.name(), "physical"): Graph,
                                         (Protein.name(), Protein.name(), "genetic"): Graph,
                                     })

Protein  nodes: 18815
Total nodes: 18815
All annotation columns (union): {'protein_size', 'annotation', 'omic', 'gene_name', 'go_id', 'Transcript sequence', 'protein_id'}
Annotation columns: ['gene_name', 'protein_size', 'protein_id', 'annotation', 'Transcript sequence', 'go_id', 'omic']
INFO: Label gene_name is transformed by MultiLabelBinarizer
INFO: Label protein_size is transformed by MultiLabelBinarizer
INFO: Label protein_id is transformed by MultiLabelBinarizer
INFO: Label annotation is split by delim '\||;' transformed by MultiLabelBinarizer
INFO: Label go_id is split by delim '\||;' transformed by MultiLabelBinarizer
INFO: Label omic is transformed by MultiLabelBinarizer


In [20]:
biogrid_edges = biogrid.get_interactions(multi_network.node_list, data=True, inclusive=True)
len(biogrid_edges)

333187

In [21]:
multi_network.add_edges([(u,v,d) for u,v,d in biogrid_edges if d["Experimental System Type"] == "physical"], 
                  database=biogrid.name(), directed=False,
                  layer=(Protein.name(), Protein.name(), "physical")
                 )
multi_network.add_edges([(u,v,d) for u,v,d in biogrid_edges if d["Experimental System Type"] == "genetic"], 
                  database=biogrid.name(), directed=False,
                  layer=(Protein.name(), Protein.name(), "genetic")
                 )

325264 edges added to self.networks[('Protein', 'Protein', 'physical')]
7923 edges added to self.networks[('Protein', 'Protein', 'genetic')]


In [22]:
with open('moge/data/gtex_biogrid_multi_network.pickle', 'wb') as file:
#     multi_network = pickle.load(file)
    pickle.dump(multi_network, file)